In [ ]:
import pandas_gbq
import pickle
import pandas as pd
import numpy as np
import re
import time
import os
import math
import random
import warnings
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from nltk.tokenize import word_tokenize
import dateutil.parser

project_id = "diesel-client-247517"

%matplotlib inline

### Pipeline for Stack Overflow Question and Answers Posts Starts Here

In [15]:
# how many total question and answer records are in Stack Overflow?
sql = """
SELECT COUNT(*) FROM `bigquery-public-data.stackoverflow.posts_questions` 
"""

numquestions = pandas_gbq.read_gbq(sql, project_id=project_id)
print("total number of questions in Stack Overflow:", numquestions)

sql = """
SELECT COUNT(*) FROM `bigquery-public-data.stackoverflow.posts_answers`
"""

numanswers = pandas_gbq.read_gbq(sql, project_id=project_id)
print("total number of answers in Stack Overflow:", numanswers)

total number of questions in Stack Overflow:         f0_
0  18154493
total number of answers in Stack Overflow:         f0_
0  27665009


In [4]:
# Here is the code to extract the records from one table and load it into a pandas data frame
# note that this can run a long time, depending on the number of records you are trying to load

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
OR tags LIKE '%graph%'
OR tags LIKE '%chart%'
OR tags LIKE '%visualiz%'
OR tags LIKE '%choropleth%'
OR tags LIKE '%drawing%'
OR tags LIKE '%line%'
OR tags LIKE '%geospatial%'
OR tags LIKE '%diagram%')
"""


In [40]:
dfPostQuestionsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['answer_count'] >= 1]

dfPostQuestionsFiltered['id'][dfPostQuestionsFiltered['accepted_answer_id'] == ''].count()

dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['accepted_answer_id'] != '']

dfPostQuestionsFiltered['new_tags']=''

In [53]:
np.shape(dfPostQuestionsFiltered)

(173698, 21)

In [23]:
# read the model tags file to be used to create the tag features from the tagtext field in the questions file, which
# has all tag values concatenated without any separators


dfModelTags = pd.read_csv("data/stackoverflow/modeltags_shortened.csv")

In [28]:
# find occurences of model tag values in tags filed, and append them to tbe new_tags field
def parsetags(qtag):
    tstr=''
    for i in dfModelTags['tagtext']:
        if i in qtag:
            tstr=tstr + i + " "
    return tstr

dfPostQuestionsFiltered['new_tags'] = dfPostQuestionsFiltered['tags'].map(parsetags)


In [32]:
# Here is a separate process to check for references to the r language in the question body
# and set a separate 'r' column in the questions df to be used to update new_tags
#
# we cannot use the same process used on the tags above, since it will almost always find occurrences of 'r'
# in the tags field that are just parts of other terms (for example, the "r" in "chart")
#
# So, here we look for standalone references to r in the question body instead
#
dfPostQuestionsFiltered['r'] = ''

def testforr(x):
    if (' r ' in x) or (' r,' in x) or (' r.' in x) or (' r:' in x):
        return 'r '
    else:
        return ''

dfPostQuestionsFiltered['r'] = dfPostQuestionsFiltered['body'].map(testforr)


In [33]:
# now, update new_tags, appending the 'r' for questions referencing r in the question body
dfPostQuestionsFiltered['new_tags'] = dfPostQuestionsFiltered['new_tags'] + dfPostQuestionsFiltered['r']

dfPostQuestionsFiltered['new_tags'][dfPostQuestionsFiltered['r']=='r ']

dfPostQuestionsFiltered['id'][dfPostQuestionsFiltered['new_tags'] == ''].count()

dfPostQuestionsFiltered = dfPostQuestionsFiltered[dfPostQuestionsFiltered['new_tags'] != '']

In [46]:
np.shape(dfPostQuestionsFiltered)

(278533, 21)

In [ ]:
# let's now create a separate question body column, 'cleaned_question_body', that removes all the code and all HTL tags
# from the question body
# We might use the cleaned_question_body as a feature in one of the similarity models
def clean_ques_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfPostQuestionsFiltered['cleaned_question_body']=''

sttime=time.time()   
 
dfPostQuestionsFiltered['cleaned_question_body'] = dfPostQuestionsFiltered['body'].map(clean_ques_body_text)


In [55]:
dfPostQuestionsFiltered['body'].head()

0     <p>I have configured and set up a fully functi...
6     <p>I want to use a mutation in an event handle...
10    <p>I have useQuery and useMutation from react-...
70    <p>I'm coding in R and I have the following da...
72    <p>I am graphing the internet connection in my...
Name: body, dtype: object

In [56]:
dfPostQuestionsFiltered['cleaned_question_body'].head()

0      I have configured and set up a fully function...
6      I want to use a mutation in an event handler....
10     I have useQuery and useMutation from react-ap...
70     I'm coding in R and I have the following data...
72     I am graphing the internet connection in my a...
Name: cleaned_question_body, dtype: object

In [101]:
# write the dataframe to a csv
dfPostQuestionsFiltered.to_csv("data/stackoverflow/PostQuestionsFiltered_V5_parsed.csv",index=False)

In [61]:
# retrieving the answers that belong to the questions retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
        OR tags LIKE '%graph%'
        OR tags LIKE '%chart%'
        OR tags LIKE '%visualiz%'
        OR tags LIKE '%choropleth%'
        OR tags LIKE '%drawing%'
        OR tags LIKE '%line%'
        OR tags LIKE '%geospatial%'
        OR tags LIKE '%diagram%'))
"""
dfPostAnswersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [62]:
np.shape(dfPostAnswersFiltered)

(545457, 20)

In [ ]:
# write the dataframe to a tsv
dfPostAnswersFiltered.to_csv("data/stackoverflow/PostAnswersFiltered_raw_12_06_19.tsv", sep='\t', index=False)

# let's drop any answers that do not have corresponding questions in our questions dataframe
tmpid=pd.DataFrame(dfPostQuestionsFiltered['id']) # temporary dataframe to hold ids from questions for merge operation
tmpid.columns=['tempqid']
dfPostAnswersFiltered=pd.merge(dfPostAnswersFiltered, tmpid, how='inner',left_on='parent_id',right_on='tempqid')

In [86]:
np.shape(dfPostAnswersFiltered)

(261052, 21)

In [89]:
# check the answer body of each answer and extract the images references into a separate column as a list
def extract_image_references(body):
    tmp_body=BeautifulSoup(body,"lxml")
    timgs=[]
    for img in tmp_body('img'):
        timgs.append(img)
    return timgs

sttime=time.time()    

dfPostAnswersFiltered['images_list']=''

dfPostAnswersFiltered['images_list'] = dfPostAnswersFiltered['body'].map(extract_image_references)
print('images finished in',(time.time()-sttime)/60,'minutes')    

images finished in 4.315905344486237 minutes


In [90]:
# check the answer body of each answer and extract the code snippets into a separate column as a list
def extract_code_snippets(body):
    tmp_body=BeautifulSoup(body,"lxml")
    tcode_snips=[]
    for code in tmp_body('code'):
        tcode_snips.append(code)
    return tcode_snips

sttime=time.time()    

dfPostAnswersFiltered['code_snippets']=''

dfPostAnswersFiltered['code_snippets'] = dfPostAnswersFiltered['body'].map(extract_code_snippets)
print('code snippets finished in',(time.time()-sttime)/60,'minutes')    

code snippets finished in 4.729599332809448 minutes


In [91]:
# create a separate 'cleaned_body' column from the answer 'body' column with all code and all html tags removed
def clean_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfPostAnswersFiltered['cleaned_body']=''

sttime=time.time()    
dfPostAnswersFiltered['cleaned_body'] = dfPostAnswersFiltered['body'].map(clean_body_text)
print('body cleaned in',(time.time()-sttime)/60,'minutes')  

body cleaned in 4.306883104642233 minutes


In [92]:
dfPostAnswersFiltered['body'].head()

0    <p>The correct syntax is this one:</p>\r\n\r\n...
1    <p>You can download the plugin manually from t...
2    <p>You can define the lines from the dataset w...
3    <p>The way matplotlib is working is that you h...
4    <p>I have figured out the answer, which is to ...
Name: body, dtype: object

In [93]:
dfPostAnswersFiltered['cleaned_body'].head()

0     The correct syntax is this one: It is working...
1     You can download the plugin manually from the...
2     You can define the lines from the dataset wit...
3     The way matplotlib is working is that you hav...
4     I have figured out the answer, which is to us...
Name: cleaned_body, dtype: object

In [95]:
dfPostAnswersFiltered['code_snippets'].head()

0    [<code>plugin install com.graphaware.es/graph-...
1                                                   []
2           [<code>mapping.get(i, "anchor"),
</code>]
3    [<code>def error_plot(ax, title, x_data, y_dat...
4    [<code>%sql select t1.time, t1.value,coalesce(...
Name: code_snippets, dtype: object

In [107]:
# add an image count column to the answers so that we can filter on answers with images
def countimages(image_list):
    if image_list == []:
        return 0
    else:
        return len(image_list)
    
dfPostAnswersFiltered['images_count']=0
dfPostAnswersFiltered['images_count'] = dfPostAnswersFiltered['images_list'].map(countimages)

In [110]:
# write the dataframe to a tsv
dfPostAnswersFiltered.to_csv("data/stackoverflow/PostAnswersFiltered_V5_parsed.tsv", sep='\t', index=False)

In [13]:
# retrieving user ids for owners of the answers retrieved above

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.users` WHERE id IN
    (SELECT owner_user_id FROM `bigquery-public-data.stackoverflow.posts_answers` WHERE parent_id IN 
        (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
            OR tags LIKE '%graph%'
            OR tags LIKE '%chart%'
            OR tags LIKE '%visualiz%'
            OR tags LIKE '%choropleth%'
            OR tags LIKE '%drawing%'
            OR tags LIKE '%line%'
            OR tags LIKE '%geospatial%'
            OR tags LIKE '%diagram%')))
"""
dfUsersFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [49]:
np.shape(dfUsersFiltered)

(304366, 13)

In [51]:
# write the dataframe to a tsv
dfUsersFiltered.to_csv("data/stackoverflow/UsersFiltered_V3.tsv", sep='\t')

In [15]:
# retrieving the comments that belong to the questions retrieved above

# NOTE - the below select may only retrieve comments related to questions??

# is a separate select needed to retrieve comments related to the answers?

sql = """
SELECT * FROM `bigquery-public-data.stackoverflow.comments` WHERE post_id IN 
    (SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions` WHERE (tags LIKE '%plot%' 
        OR tags LIKE '%graph%'
        OR tags LIKE '%chart%'
        OR tags LIKE '%visualiz%'
        OR tags LIKE '%choropleth%'
        OR tags LIKE '%drawing%'
        OR tags LIKE '%line%'
        OR tags LIKE '%geospatial%'
        OR tags LIKE '%diagram%'))
"""
dfCommentsFiltered = pandas_gbq.read_gbq(sql, project_id=project_id)

In [16]:
np.shape(dfCommentsFiltered)

(1184056, 7)

In [18]:
# write the dataframe to a tsv
dfCommentsFiltered.to_csv("data/stackoverflow/CommentsFiltered_v3.tsv", sep='\t')

### Pipeline for Stack Exchange Data Science Community Question and Answers Posts Starts Here

In [115]:
# Use this code to load raw Data Science Stack Exchange questions and answers to Pandas dfs -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to

dfDSPosts = pd.read_csv("Data/Stack Exchange Data Science/Posts_From_Data_Science_Stack_Exchange.csv", keep_default_na=False)

dfDSQues=dfDSPosts[dfDSPosts['PostTypeId']==1]
dfDSAns=dfDSPosts[dfDSPosts['PostTypeId']==2]

del dfDSPosts

In [120]:
np.shape(dfDSQues)

(20502, 22)

In [121]:
np.shape(dfDSAns)

(22666, 22)

In [140]:

dfDSQues['AnswerCount'][dfDSQues['AnswerCount'] != '0'].count()

dfDSQues = dfDSQues[dfDSQues['AnswerCount'] != '0' ]

dfDSQues = dfDSQues[dfDSQues['AcceptedAnswerId'] != '']

In [141]:
np.shape(dfDSQues)

(6694, 22)

In [142]:
# read the model tags file to be used to create the tag features from the tagtext field in the questions file, which
# has all tag values concatenated without any separators

# NOTE: Used shortened version of model tag list below

dfModelTags = pd.read_csv("data/stackoverflow/modeltags_shortened.csv")

In [ ]:
# find occurences of model tag values in tags filed, and append them to tbe new_tags field
def parsetags(qtag):
    tstr=''
    for i in dfModelTags['tagtext']:
        if i in qtag:
            tstr=tstr + i + " "
    return tstr

dfDSQues['new_tags'] = dfDSQues['Tags'].map(parsetags)

In [150]:
# Here is a separate process to check for references to the r language in the question body
# and set a separate 'r' column in the questions df to be used to update new_tags
#
# we cannot use the same process used on the tags above, since it will almost always find occurrences of 'r'
# in the tags field that are just parts of other terms (for example, the "r" in "chart")
#
# So, here we look for standalone references to r in the question body instead
#
dfDSQues['r'] = ''

def testforr(x):
    if (' r ' in x) or (' r,' in x) or (' r.' in x) or (' r:' in x):
        return 'r '
    else:
        return ''

dfDSQues['r'] = dfDSQues['Body'].map(testforr)


In [156]:
# now, update new_tags, appending the 'r' for questions referencing r in the question body
dfDSQues['new_tags'] = dfDSQues['new_tags'] + dfDSQues['r']

# now, let's drop any questions that do not have tags we are interested in

dfDSQues['Id'][dfDSQues['new_tags'] == ''].count()

5109

In [157]:
dfDSQues = dfDSQues[dfDSQues['new_tags'] != '']

In [158]:
np.shape(dfDSQues)

(1585, 23)

In [ ]:
# let's now create a separate question body column, 'cleaned_question_body', that removes all the code and all HTL tags
# from the question body
# We might use the cleaned_question_body as a feature in one of the similarity models
def clean_ques_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfDSQues['cleaned_question_body']=''

sttime=time.time()    
dfDSQues['cleaned_question_body'] = dfDSQues['Body'].map(clean_ques_body_text)


In [160]:
dfDSQues['Body'].head()

53    <p>I have a excel data with time stamp format ...
56    <p>I am learning <a href="https://en.wikipedia...
65    <p>I have trained an ANN model for a regressio...
84    <p>I'm getting confused as how to proceed for ...
90    <p>I am learning <a href="https://en.wikipedia...
Name: Body, dtype: object

In [161]:
dfDSQues['cleaned_question_body'].head()

53     I have a excel data with time stamp format li...
56     I am learning SVD by following this MIT cours...
65     I have trained an ANN model for a regression ...
84     I'm getting confused as how to proceed for th...
90     I am learning SVD by following this MIT cours...
Name: cleaned_question_body, dtype: object

In [164]:
# write the dataframe to a tsv
dfDSQues.to_csv("data/stackoverflow/DataSciQues_parsed.tsv", sep='\t', index=False)

# write the dataframe to a csv
dfDSQues.to_csv("data/stackoverflow/DataSciQues_parsed.csv",index=False)

In [181]:
# let's drop any answers that do not have corresponding questions in our questions dataframe
tmpid=pd.DataFrame(dfDSQues['Id']) # temporary dataframe to hold ids from questions for merge operation
tmpid.columns=['tempqid']
dfDSAns=pd.merge(dfDSAns, tmpid, how='inner',left_on=pd.to_numeric(dfDSAns['ParentId'], downcast='integer'),right_on='tempqid')
del dfDSAns['tempqid']
del tmpid

In [183]:
np.shape(dfDSAns)

(2410, 22)

In [184]:
# check the answer body of each answer and extract the images references into a separate column as a list
def extract_image_references(body):
    tmp_body=BeautifulSoup(body,"lxml")
    timgs=[]
    for img in tmp_body('img'):
        timgs.append(img)
    return timgs

sttime=time.time()    

dfDSAns['images_list']=''

dfDSAns['images_list'] = dfDSAns['Body'].map(extract_image_references)
print('images finished in',(time.time()-sttime)/60,'minutes')    

images finished in 0.018283510208129884 minutes


In [185]:
# check the answer body of each answer and extract the code snippets into a separate column as a list
def extract_code_snippets(body):
    tmp_body=BeautifulSoup(body,"lxml")
    tcode_snips=[]
    for code in tmp_body('code'):
        tcode_snips.append(code)
    return tcode_snips

sttime=time.time()    

dfDSAns['code_snippets']=''

dfDSAns['code_snippets'] = dfDSAns['Body'].map(extract_code_snippets)
print('code snippets finished in',(time.time()-sttime)/60,'minutes')    

code snippets finished in 0.0185501495997111 minutes


In [187]:
# create a separate 'cleaned_body' column from the answer 'body' column with all code and all html tags removed
def clean_body_text(body):
    tmp_body=BeautifulSoup(body,"lxml")
    for code in tmp_body('code'):
        code.replace_with('')
    tstr=''
    for string in tmp_body.stripped_strings:
        tstr = tstr + ' ' + string
    return tstr

dfDSAns['cleaned_body']=''

sttime=time.time()    
dfDSAns['cleaned_body'] = dfDSAns['Body'].map(clean_body_text)
print('body cleaned in',(time.time()-sttime)/60,'minutes')  

body cleaned in 0.01733266512552897 minutes


In [188]:
dfDSAns['Body'].head()

0    <p>As We should not remove any data ... we can...
1    <p>Consider using the <a href="https://en.wiki...
2    <p>You could take an <a href="https://en.wikip...
3    <p>Did you try keeping a separate test set (on...
4    <p>From the curves you are showing <strong>yes...
Name: Body, dtype: object

In [189]:
dfDSAns['cleaned_body'].head()

0     As We should not remove any data ... we can u...
1     Consider using the Earth Mover's Distance (i....
2     You could take an Information Theory approach...
3     Did you try keeping a separate test set (on t...
4     From the curves you are showing yes . Over-fi...
Name: cleaned_body, dtype: object

In [192]:
# add an image count column to the answers so that we can filter on answers with images
def countimages(image_list):
    if image_list == []:
        return 0
    else:
        return len(image_list)
    
dfDSAns['images_count']=0
dfDSAns['images_count'] = dfDSAns['images_list'].map(countimages)

In [196]:
# write the dataframe to a tsv
dfDSAns.to_csv("data/stackoverflow/DataSciAns_parsed.tsv", sep='\t', index=False)

In [22]:
# USE THIS CODE TO LOAD THE DATA SCIENCE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfDSAns = pd.read_csv("data/stackoverflow/DataSciAns_parsed.tsv", sep='\t', keep_default_na=False)

In [24]:
# let's check to see if there duplicate ids in the Stack Overflow and Data Science Stack Exchange datasets before
# combining them
tsoquesids=pd.DataFrame(dfPostQuestionsFiltered['id'])
tsoansids=pd.DataFrame(dfPostAnswersFiltered['id'])
tdsquesids=pd.DataFrame(dfDSQues['Id'])
tdsansids=pd.DataFrame(dfDSAns['Id'])
tmergeques=pd.merge(tsoquesids, tdsquesids, how='inner', left_on='id', right_on='Id')
tmergeans=pd.merge(tsoansids, tdsansids, how='inner', left_on='id', right_on='Id')

In [28]:
# dropping questions with duplicate ids from the Stack Overflow and data science question sets
# also dropping answers that refer back to those duplicate questions
# note: we do not drop answers that happen to have duplicate ids as the risk of getting bad return results
# in our models is minimal for duplicate answers, espcially given the small number of duplicate answer ids
qidstodrop=tmergeques['Id'].values.tolist()
print('duplicate question Ids to be dropped:',qidstodrop)

duplicate question Ids to be dropped: [58649]


In [33]:
for dupqid in qidstodrop:
    
    SOanswithdupqids=dfPostAnswersFiltered['id'][dfPostAnswersFiltered['parent_id']==dupqid].index
    print('dropping',len(SOanswithdupqids),'answers from SO answers that have question parent id =',dupqid)
    dfPostAnswersFiltered.drop(labels=SOanswithdupqids,inplace=True)
    
    DSanswithdupqids=dfDSAns['Id'][dfDSAns['ParentId']==dupqid].index
    print('dropping',len(DSanswithdupqids),'answers from DS answers that have question parent id =',dupqid)
    dfDSAns.drop(labels=DSanswithdupqids,inplace=True)

    print('dropping quesion from SO that has question =',dupqid)
    dfPostQuestionsFiltered.drop(labels=dfPostQuestionsFiltered[dfPostQuestionsFiltered['id']==dupqid].index,inplace=True)

    print('dropping quesion from DS that has question =',dupqid)
    dfDSQues.drop(labels=dfDSQues[dfDSQues['Id']==dupqid].index,inplace=True)
    

dropping 7 answers from SO answers that have question parent id = 58649
dropping 1 answers from DS answers that have question parent id = 58649
dropping quesion from SO that has question = 58649
dropping quesion from DS that has question = 58649


In [34]:
print('length of dataframes after dropping duplicate question Ids -- SO ques:',len(dfPostQuestionsFiltered), 'DS ques', len(dfDSQues),'SO ans:',len(dfPostAnswersFiltered), 'DS ans', len(dfDSAns) )

length of dataframes after dropping duplicate question Ids -- SO ques: 173697 DS ques 1584 SO ans: 261045 DS ans 2409


In [35]:
dfPostAnswersFiltered.columns

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'parent_id', 'post_type_id', 'score', 'tags',
       'view_count', 'images_list', 'code_snippets', 'cleaned_body',
       'images_count'],
      dtype='object')

In [38]:
# change the column names in data science answers dataset to match those in the Stack Overflow answers dataset
dfDSAns.columns = ['id', 'post_type_id', 'accepted_answer_id', 'parent_id', 'creation_date', 'score', 
                   'view_count', 'body', 'owner_user_id', 'owner_display_name', 'last_editor_user_id', 
                   'last_editor_display_name', 'last_edit_date', 'last_activity_date', 'title', 'tags',
                   'answer_count', 'comment_count', 'favorite_count', 'community_owned_date', 
                   'images_list', 'code_snippets', 'cleaned_body', 'images_count']

In [ ]:
dfCombinedAns=dfPostAnswersFiltered.append(dfDSAns,ignore_index=True)

In [41]:
np.shape(dfPostAnswersFiltered)

(261045, 24)

In [42]:
np.shape(dfDSAns)

(2409, 24)

In [43]:
np.shape(dfCombinedAns)

(263454, 24)

In [48]:
# change the column names in data science answers dataset to match those in the Stack Overflow answers dataset
dfDSQues.columns = ['id', 'post_type_id', 'accepted_answer_id', 'parent_id', 'creation_date', 'score', 
                   'view_count', 'body', 'owner_user_id', 'owner_display_name', 'last_editor_user_id', 
                   'last_editor_display_name', 'last_edit_date', 'last_activity_date', 'title', 'tags',
                   'answer_count', 'comment_count', 'favorite_count', 'community_owned_date', 
                   'new_tags', 'cleaned_question_body']

In [ ]:
dfCombinedQues=dfPostQuestionsFiltered.append(dfDSQues,ignore_index=True)

In [51]:
np.shape(dfPostQuestionsFiltered)

(173697, 22)

In [52]:
np.shape(dfDSQues)

(1584, 22)

In [53]:
np.shape(dfCombinedQues)

(175281, 22)

In [54]:
# write the combined questions dataframe to a tsv
dfCombinedQues.to_csv("data/stackoverflow/SODSQues_parsed.tsv", sep='\t', index=False)

In [55]:
# write the combined questions dataframe to a csv
dfCombinedQues.to_csv("data/stackoverflow/SODSQues_parsed.csv", index=False)

In [22]:
# USE THIS CODE TO LOAD THE SO + DATA SCIENCE QUESTIONS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCombinedQues = pd.read_csv("data/stackoverflow/SODSQues_parsed.tsv", sep='\t', keep_default_na=False)

In [56]:
# write the combined answers dataframe to a tsv
dfCombinedAns.to_csv("data/stackoverflow/SODSAns_parsed.tsv", sep='\t', index=False)

In [22]:
# USE THIS CODE TO LOAD THE SO + DATA SCIENCE ANSWERS TO A PANDAS DATAFRAME -- Note that you may have to change the path to the .tsv 
# file in the statement below depending on the location you downladed the .tsv to
dfCombinedAns = pd.read_csv("data/stackoverflow/SODSAns_parsed.tsv", sep='\t', keep_default_na=False)